In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.constraints import max_norm


In [2]:
my_gpu = tf.config.experimental.list_physical_devices('GPU')
print(my_gpu)
tf.config.experimental.set_memory_growth(my_gpu[0],True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
df = pd.read_csv('final_data.csv')

df.drop(['href', 'href','stress_address','city','state','county','img_href',"zip"], axis=1, inplace=True)
dfX = df.drop("price",axis=1)
dfX

,beds,bath,area,year_built,lot_size,walk_score,transit_score,bike_score,cooling,heating,has_pool,crime_rate_per_100000,minimum_wage,college_completion,unemployment_rate,median_household_income
0,3,2.0,1229,1983,0,68,39,63,0,0,0,442.965072,7.25,0.315,7.7,61807.0
1,3,1.0,1348,1913,0,70,0,46,0,0,0,102.396264,7.25,0.413,8.3,93587.0
2,3,1.0,1434,1941,0,45,19,39,1,2,0,177.018370,7.25,0.437,6.5,68975.0
3,3,2.0,1356,1961,0,11,15,33,2,2,0,721.521934,7.25,0.290,9.3,44773.0
4,5,1.5,2266,1940,0,5,0,42,0,3,0,18.351570,8.70,0.097,3.9,62596.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36330,2,1.0,928,1962,0,47,0,35,0,3,0,89.855758,11.25,0.194,8.2,52543.0
36331,3,1.5,1454,1956,0,42,0,44,0,0,0,290.721821,9.65,0.249,12.0,65051.0
36332,4,2.5,1772,2006,0,1,24,35,1,2,0,738.481892,9.00,0.344,8.1,56148.0
36333,3,1.5,1706,1930,0,55,0,46,0,3,0,521.234940,8.70,0.325,10.4,52503.0


In [4]:
X,y = np.array(dfX),np.array(df["price"])
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
model = Sequential([
    Dense(units=128,input_shape=(len(X[0]),),kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=256,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation = LeakyReLU(alpha=0.01)),
    Dense(units=1,kernel_initializer=initializers.RandomNormal(stddev=0.01),activation =  LeakyReLU(alpha=0.01))
])

In [6]:
def coeff_determination(y_true, y_pred):
    from tensorflow.keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [7]:
convergeAlarm = EarlyStopping(monitor='loss', min_delta=0, patience=50)
def lr_scheduler(epoch, lr):
    if epoch % 15 == 0 and epoch:
        return lr /1.15
    return lr

In [8]:
model.compile(optimizer= SGD(learning_rate=0.0001, momentum=0.9), loss='mean_absolute_error', metrics = [coeff_determination])
history = model.fit(x=X_train, y=y_train, batch_size=128, epochs=40000, verbose=2,validation_split=0.2,shuffle=True, callbacks=[convergeAlarm,LearningRateScheduler(lr_scheduler, verbose=1)])

Epoch 1/40000

Epoch 00001: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
182/182 - 1s - loss: 630770.7500 - coeff_determination: -5.4321e-01 - val_loss: 646208.0000 - val_coeff_determination: -5.9180e-01
Epoch 2/40000

Epoch 00002: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
182/182 - 0s - loss: 630770.6250 - coeff_determination: -5.4767e-01 - val_loss: 646207.6875 - val_coeff_determination: -5.9180e-01
Epoch 3/40000

Epoch 00003: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
182/182 - 0s - loss: 630770.3125 - coeff_determination: -5.4634e-01 - val_loss: 646207.5625 - val_coeff_determination: -5.9180e-01
Epoch 4/40000

Epoch 00004: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
182/182 - 0s - loss: 630769.9375 - coeff_determination: -5.3821e-01 - val_loss: 646207.4375 - val_coeff_determination: -5.9180e-01
Epoch 5/40000

Epoch 00005: LearningRateScheduler reducing learning rate to 9.99

Epoch 37/40000

Epoch 00037: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/182 - 0s - loss: 298266.2188 - coeff_determination: 0.3646 - val_loss: 316865.6875 - val_coeff_determination: 0.1488
Epoch 38/40000

Epoch 00038: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/182 - 0s - loss: 321034.2500 - coeff_determination: 0.3285 - val_loss: 523190.2500 - val_coeff_determination: -8.3174e+00
Epoch 39/40000

Epoch 00039: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/182 - 0s - loss: 318057.1875 - coeff_determination: 0.2261 - val_loss: 298710.8125 - val_coeff_determination: 0.0359
Epoch 40/40000

Epoch 00040: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/182 - 0s - loss: 305751.6875 - coeff_determination: 0.3768 - val_loss: 313203.7500 - val_coeff_determination: -5.8634e-02
Epoch 41/40000

Epoch 00041: LearningRateScheduler reducing learning rate to 7.561436359537765e-05.
182/18

Epoch 74/40000

Epoch 00074: LearningRateScheduler reducing learning rate to 5.717532258131541e-05.
182/182 - 0s - loss: 274696.0000 - coeff_determination: 0.4170 - val_loss: 310671.6875 - val_coeff_determination: 0.1821
Epoch 75/40000

Epoch 00075: LearningRateScheduler reducing learning rate to 5.717532258131541e-05.
182/182 - 0s - loss: 277288.0312 - coeff_determination: 0.3532 - val_loss: 284774.7188 - val_coeff_determination: 0.4445
Epoch 76/40000

Epoch 00076: LearningRateScheduler reducing learning rate to 4.9717671809839494e-05.
182/182 - 0s - loss: 264504.2188 - coeff_determination: 0.4875 - val_loss: 287072.1562 - val_coeff_determination: 0.4051
Epoch 77/40000

Epoch 00077: LearningRateScheduler reducing learning rate to 4.9717673391569406e-05.
182/182 - 0s - loss: 268570.5938 - coeff_determination: 0.4625 - val_loss: 282376.3125 - val_coeff_determination: 0.4345
Epoch 78/40000

Epoch 00078: LearningRateScheduler reducing learning rate to 4.9717673391569406e-05.
182/182 - 1s 

182/182 - 0s - loss: 240257.0938 - coeff_determination: 0.5591 - val_loss: 263994.0938 - val_coeff_determination: 0.3504
Epoch 112/40000

Epoch 00112: LearningRateScheduler reducing learning rate to 3.759370156330988e-05.
182/182 - 0s - loss: 244825.0000 - coeff_determination: 0.4396 - val_loss: 266609.2500 - val_coeff_determination: 0.4687
Epoch 113/40000

Epoch 00113: LearningRateScheduler reducing learning rate to 3.759370156330988e-05.
182/182 - 0s - loss: 238129.3906 - coeff_determination: 0.5852 - val_loss: 260262.1719 - val_coeff_determination: 0.4880
Epoch 114/40000

Epoch 00114: LearningRateScheduler reducing learning rate to 3.759370156330988e-05.
182/182 - 0s - loss: 239446.5000 - coeff_determination: 0.5667 - val_loss: 266572.7188 - val_coeff_determination: 0.4588
Epoch 115/40000

Epoch 00115: LearningRateScheduler reducing learning rate to 3.759370156330988e-05.
182/182 - 0s - loss: 238781.7031 - coeff_determination: 0.5463 - val_loss: 265883.6875 - val_coeff_determination

182/182 - 0s - loss: 233964.7188 - coeff_determination: -4.1034e-02 - val_loss: 269590.0312 - val_coeff_determination: 0.4032
Epoch 149/40000

Epoch 00149: LearningRateScheduler reducing learning rate to 2.8426238714018837e-05.
182/182 - 0s - loss: 229052.5312 - coeff_determination: 0.5766 - val_loss: 267254.6562 - val_coeff_determination: 0.4168
Epoch 150/40000

Epoch 00150: LearningRateScheduler reducing learning rate to 2.8426238714018837e-05.
182/182 - 0s - loss: 227738.5000 - coeff_determination: 0.5773 - val_loss: 271739.3438 - val_coeff_determination: 0.3738
Epoch 151/40000

Epoch 00151: LearningRateScheduler reducing learning rate to 2.4718468446972903e-05.
182/182 - 0s - loss: 225268.5781 - coeff_determination: 0.6140 - val_loss: 262842.7500 - val_coeff_determination: 0.4461
Epoch 152/40000

Epoch 00152: LearningRateScheduler reducing learning rate to 2.4718468921491876e-05.
182/182 - 0s - loss: 220042.0781 - coeff_determination: 0.6326 - val_loss: 278853.3125 - val_coeff_dete

...

182/182 - 0s - loss: 143661.0781 - coeff_determination: 0.7978 - val_loss: 268252.0000 - val_coeff_determination: 0.2901
Epoch 371/40000

Epoch 00371: LearningRateScheduler reducing learning rate to 3.493428494039108e-06.
182/182 - 0s - loss: 140968.5312 - coeff_determination: 0.8037 - val_loss: 272177.5938 - val_coeff_determination: 0.2827
Epoch 372/40000

Epoch 00372: LearningRateScheduler reducing learning rate to 3.493428494039108e-06.
182/182 - 0s - loss: 140215.8906 - coeff_determination: 0.7936 - val_loss: 271663.0625 - val_coeff_determination: 0.2808
Epoch 373/40000

Epoch 00373: LearningRateScheduler reducing learning rate to 3.493428494039108e-06.
182/182 - 0s - loss: 141902.3125 - coeff_determination: 0.8033 - val_loss: 274409.0000 - val_coeff_determination: 0.2042
Epoch 374/40000

Epoch 00374: LearningRateScheduler reducing learning rate to 3.493428494039108e-06.
182/182 - 1s - loss: 140749.3125 - coeff_determination: 0.8158 - val_loss: 271136.6250 - val_coeff_determination

182/182 - 0s - loss: 130605.1797 - coeff_determination: 0.8214 - val_loss: 267923.8438 - val_coeff_determination: 0.3814
Epoch 408/40000

Epoch 00408: LearningRateScheduler reducing learning rate to 2.2969859401200665e-06.
182/182 - 0s - loss: 131507.6094 - coeff_determination: 0.8226 - val_loss: 274757.3125 - val_coeff_determination: 0.2672
Epoch 409/40000

Epoch 00409: LearningRateScheduler reducing learning rate to 2.2969859401200665e-06.
182/182 - 0s - loss: 130219.9062 - coeff_determination: 0.8139 - val_loss: 273113.7500 - val_coeff_determination: 0.2979
Epoch 410/40000

Epoch 00410: LearningRateScheduler reducing learning rate to 2.2969859401200665e-06.
182/182 - 0s - loss: 130099.7734 - coeff_determination: 0.8157 - val_loss: 276223.5312 - val_coeff_determination: 0.2379
Epoch 411/40000

Epoch 00411: LearningRateScheduler reducing learning rate to 2.2969859401200665e-06.
182/182 - 1s - loss: 130071.7344 - coeff_determination: 0.8173 - val_loss: 272992.3125 - val_coeff_determina

182/182 - 0s - loss: 124494.4453 - coeff_determination: 0.8171 - val_loss: 272466.5938 - val_coeff_determination: 0.3123
Epoch 445/40000

Epoch 00445: LearningRateScheduler reducing learning rate to 1.7368513454130152e-06.
182/182 - 0s - loss: 124826.6641 - coeff_determination: 0.8301 - val_loss: 275219.8125 - val_coeff_determination: 0.2960
Epoch 446/40000

Epoch 00446: LearningRateScheduler reducing learning rate to 1.7368513454130152e-06.
182/182 - 0s - loss: 126014.8203 - coeff_determination: 0.8247 - val_loss: 273579.1875 - val_coeff_determination: 0.3021
Epoch 447/40000

Epoch 00447: LearningRateScheduler reducing learning rate to 1.7368513454130152e-06.
182/182 - 0s - loss: 124463.7969 - coeff_determination: 0.8233 - val_loss: 276264.6250 - val_coeff_determination: 0.2681
Epoch 448/40000

Epoch 00448: LearningRateScheduler reducing learning rate to 1.7368513454130152e-06.
182/182 - 0s - loss: 123971.5469 - coeff_determination: 0.8261 - val_loss: 272685.7500 - val_coeff_determina

182/182 - 1s - loss: 118838.0859 - coeff_determination: 0.8456 - val_loss: 276525.1250 - val_coeff_determination: 0.2659
Epoch 482/40000

Epoch 00482: LearningRateScheduler reducing learning rate to 1.1420080454627168e-06.
182/182 - 0s - loss: 117755.3203 - coeff_determination: 0.8321 - val_loss: 272927.6562 - val_coeff_determination: 0.3278
Epoch 483/40000

Epoch 00483: LearningRateScheduler reducing learning rate to 1.1420080454627168e-06.
182/182 - 0s - loss: 118355.3516 - coeff_determination: 0.8250 - val_loss: 276318.4688 - val_coeff_determination: 0.2900
Epoch 484/40000

Epoch 00484: LearningRateScheduler reducing learning rate to 1.1420080454627168e-06.
182/182 - 0s - loss: 117746.5547 - coeff_determination: 0.8346 - val_loss: 276481.0625 - val_coeff_determination: 0.2866
Epoch 485/40000

Epoch 00485: LearningRateScheduler reducing learning rate to 1.1420080454627168e-06.
182/182 - 0s - loss: 118046.4375 - coeff_determination: 0.8370 - val_loss: 275038.6562 - val_coeff_determina

KeyboardInterrupt: 